In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim

import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools

#from nltk.stem.snowball import SnowballStemmer
import spacy

from multiprocessing import Pool, cpu_count

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Splitting speeches by Parties and time

In [21]:
%%time
#with open('/kaggle/input/dfforlemmatization/dfForLemzn (1).pkl', 'rb') as f:
with open('./dfForLemzn.pkl', 'rb') as f:

    partyTimeDf = pickle.load(f)
    
#print(partyTimeDf)
spacyMod = spacy.load('en_core_web_sm')
spacyMod.max_length =104584200
#17417090
#1.37*10000000 


partyTimeDf['Lemmatized']='s'
orig =partyTimeDf


CPU times: user 470 ms, sys: 301 ms, total: 771 ms
Wall time: 769 ms


In [22]:
print('Hello')

Hello


In [23]:
partyTimeDf.LengthLemmas.describe()

count    3.000000e+01
mean     1.420904e+06
std      3.151005e+06
min      7.230000e+02
25%      2.277200e+04
50%      1.590335e+05
75%      7.488518e+05
max      1.373554e+07
Name: LengthLemmas, dtype: float64

In [24]:
partyTimeDf

,party,df_name,sentence,LengthLemmas,Lemmatized
0,Democratic Unionist Party,df_t1_Democratic Unionist Party,join prime minister leader opposition paying t...,358527,s
0,Democratic Unionist Party,df_t2_Democratic Unionist Party,prime minister different sides argument spoke ...,274888,s
0,Liberal Democrat,df_t1_Liberal Democrat,delighted follow right member hitchin harpende...,2272943,s
0,Liberal Democrat,df_t2_Liberal Democrat,thank veterans minister taking account plight ...,303518,s
0,Independent,df_t1_Independent,congratulate member maidstone weald mrs grant ...,65153,s
0,Independent,df_t2_Independent,grade listed rochdale town hall described poss...,116176,s
0,Labour,df_t1_Labour,sure whole house join paying tribute know prim...,8249528,s
0,Labour,df_t2_Labour,given intensified bombing aleppo president put...,3862143,s
0,UK Independence Party,df_t1_UK Independence Party,friend member richmond park zac goldsmith pres...,12557,s
0,UK Independence Party,df_t2_UK Independence Party,applaud prime minister welcome statement withd...,723,s


In [25]:
partyTimeDf.reset_index(inplace=True)
partyTimeDf

,index,party,df_name,sentence,LengthLemmas,Lemmatized
0,0,Democratic Unionist Party,df_t1_Democratic Unionist Party,join prime minister leader opposition paying t...,358527,s
1,0,Democratic Unionist Party,df_t2_Democratic Unionist Party,prime minister different sides argument spoke ...,274888,s
2,0,Liberal Democrat,df_t1_Liberal Democrat,delighted follow right member hitchin harpende...,2272943,s
3,0,Liberal Democrat,df_t2_Liberal Democrat,thank veterans minister taking account plight ...,303518,s
4,0,Independent,df_t1_Independent,congratulate member maidstone weald mrs grant ...,65153,s
5,0,Independent,df_t2_Independent,grade listed rochdale town hall described poss...,116176,s
6,0,Labour,df_t1_Labour,sure whole house join paying tribute know prim...,8249528,s
7,0,Labour,df_t2_Labour,given intensified bombing aleppo president put...,3862143,s
8,0,UK Independence Party,df_t1_UK Independence Party,friend member richmond park zac goldsmith pres...,12557,s
9,0,UK Independence Party,df_t2_UK Independence Party,applaud prime minister welcome statement withd...,723,s


In [26]:
firstHalf = partyTimeDf.iloc[:int(partyTimeDf.shape[0]/2)]
secondHalf = partyTimeDf.iloc[int(partyTimeDf.shape[0]/2):int(partyTimeDf.shape[0])]

In [27]:
firstHalf

,index,party,df_name,sentence,LengthLemmas,Lemmatized
0,0,Democratic Unionist Party,df_t1_Democratic Unionist Party,join prime minister leader opposition paying t...,358527,s
1,0,Democratic Unionist Party,df_t2_Democratic Unionist Party,prime minister different sides argument spoke ...,274888,s
2,0,Liberal Democrat,df_t1_Liberal Democrat,delighted follow right member hitchin harpende...,2272943,s
3,0,Liberal Democrat,df_t2_Liberal Democrat,thank veterans minister taking account plight ...,303518,s
4,0,Independent,df_t1_Independent,congratulate member maidstone weald mrs grant ...,65153,s
5,0,Independent,df_t2_Independent,grade listed rochdale town hall described poss...,116176,s
6,0,Labour,df_t1_Labour,sure whole house join paying tribute know prim...,8249528,s
7,0,Labour,df_t2_Labour,given intensified bombing aleppo president put...,3862143,s
8,0,UK Independence Party,df_t1_UK Independence Party,friend member richmond park zac goldsmith pres...,12557,s
9,0,UK Independence Party,df_t2_UK Independence Party,applaud prime minister welcome statement withd...,723,s


In [28]:
secondHalf

,index,party,df_name,sentence,LengthLemmas,Lemmatized
15,0,Speaker,df_t1_Speaker,acquaint house house day attended majesty hous...,265069,s
16,0,Speaker,df_t2_Speaker,arithmetic may faulty counted six questions kn...,278044,s
17,0,Labour (Co-op),df_t1_Labour (Co-op),would grateful prime minister could confirm go...,1221423,s
18,0,Labour (Co-op),df_t2_Labour (Co-op),whoever stands opposition dispatch box governm...,748272,s
19,0,Conservative,df_t1_Conservative,beg move humble address presented majesty foll...,13735544,s
20,0,Conservative,df_t2_Conservative,hope allow mr speaker add tribute jo cox work ...,8178395,s
21,0,Green Party,df_t1_Green Party,right friend says challenge climate change gre...,106889,s
22,0,Green Party,df_t2_Green Party,prime minister recognise whoever becomes next ...,38402,s
23,0,The Independent Group for Change,df_t2_The Independent Group for Change,speculation long kim darroch remain post given...,10129,s
24,0,Scottish National Party,df_t1_Scottish National Party,know liberals quite readily abandon plea propo...,749045,s


In [29]:
def parallelize_dataframe(df, func, num_partitions):
    print(0)
    df_split = np.array_split(df, num_partitions)
    #print(df_split)
    #print(1)
    pool = Pool(num_partitions)
    #print(2)
    temp= pool.map(func, df_split)

    print('Returned to be now concatenated', len(temp))
    df = pd.concat(temp)
    #print('1')
    pool.close()
    pool.join()
    return df
#ERROR! Session/line number was not unique in database. History logging moved to new session 2


In [ ]:
%%time
#count=0
def my_func(df):
    print('Been called')
    #print(count)
    #count=count+1
    #print(df.shape)[0]
    
    if(df.shape[0]==0):
        print('Empty df gotten')
        
    elif(df.shape[0]==1):
        df["lemmatized"] = df['sentence'].apply(lambda x:[y.lemma_ for y in spacyMod(x, disable = ['ner','parser'])])
        print('returning',df['df_name'], 'lemmatizing')
        
        dfPath = './' + str(df.iloc[0]['df_name'])+'.pkl'
        print('df path is -',dfPath)
        df.to_pickle(dfPath)
    
        
    elif(df.shape[0]>1):
        print('DF rows more than one, DF is - ', df)
    
    else:
        print('Seems like series gotten. Who are you?',df)
        
        
    #exit()
    return df

num_cores = cpu_count()
print('Cores',num_cores)
ress = parallelize_dataframe(firstLabour, my_func, 2)
#num_cores

Cores 20
0
Been called
Empty df gotten
Been called


In [16]:
#17 till df_t2_speaker
#ress
#29106189

labours = firstHalf.iloc[6:8]
#labours
firstLabour = labours.iloc[[0]]


conservatives = secondHalf.iloc[[4]] #df t1 Conservatives
#type(conservatives)

In [41]:
print(haw)

In [36]:


#with open('./4 df_t1_Independent Name: df_name, dtype: object.pkl', 'rb') as f:
with open('./df_t2_Speaker.pkl', 'rb') as f:
    print('d')
    tst = pickle.load(f)
print('Hello',tst)

In [ ]:
ress.to_pickle('./df1to2.pkl')
#ress.to_pickle('./df1to2.pkl')

In [21]:
partyTimeDf

,party,df_name,sentence,LengthLemmas,Lemmatized
0,Democratic Unionist Party,df_t1_Democratic Unionist Party,join prime minister leader opposition paying t...,358527,s
0,Democratic Unionist Party,df_t2_Democratic Unionist Party,prime minister different sides argument spoke ...,274888,s
0,Liberal Democrat,df_t1_Liberal Democrat,delighted follow right member hitchin harpende...,2272943,s
0,Liberal Democrat,df_t2_Liberal Democrat,thank veterans minister taking account plight ...,303518,s
0,Independent,df_t1_Independent,congratulate member maidstone weald mrs grant ...,65153,s
0,Independent,df_t2_Independent,grade listed rochdale town hall described poss...,116176,s
0,Labour,df_t1_Labour,sure whole house join paying tribute know prim...,8249528,s
0,Labour,df_t2_Labour,given intensified bombing aleppo president put...,3862143,s
0,UK Independence Party,df_t1_UK Independence Party,friend member richmond park zac goldsmith pres...,12557,s
0,UK Independence Party,df_t2_UK Independence Party,applaud prime minister welcome statement withd...,723,s


In [ ]:
%%time
#tokens =[token for token in pDf['Lemmas']][0]

spacyMod = spacy.load('en_core_web_sm')
spacyMod.max_length = 1.39*10000000 
    
    #Disabling RaM hungry parts not needed for lemmatization
    #lemmatizedSent = spacyMod(sent, disable = ['ner', 'parser'])
    #lemmatizedSent = [word.lemma_ for word in lemmatizedSent]
    #print(len(tokens))

partyTimeDf["lemmatized"] = partyTimeDf['sentence'].apply(lambda x: " ".join([y.lemma_ for y in spacyMod.pipe(x, disable = ['ner','parser'])]))
    

In [ ]:
%%time


# Use English stemmer.
stemmer = SnowballStemmer("english")
spacyMod = spacy.load('en_core_web_sm')
spacyMod.max_length = 1.39*10000000 


for pDf in dictSpeechesByParty.values():
    
    tokens =[token for token in pDf['Lemmas']][0]
    sent =' '.join(tokens)

    pDf['sentence'] =sent
    
    #Disabling RaM hungry parts not needed for lemmatization
    #lemmatizedSent = spacyMod(sent, disable = ['ner', 'parser'])
    #lemmatizedSent = [word.lemma_ for word in lemmatizedSent]
    print(len(tokens))

    pDf['stemmed'] = pDf['Lemmas'].apply(lambda x: [stemmer.stem(y) for y in x])
    #pDf['lemmatized'] = [lemmatizedSent]

#ValueError: [E088] Text of length 9266826 exceeds maximum of 1000000. 
#The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. 
#This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. 
#The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.
'''Done till here when both stemming and lemmatization done- 65156
116180
5340
156935
80876
10129
749124'''
#, 9.8 GBs used

In [ ]:
'''%%time


# Use English stemmer.
stemmer = SnowballStemmer("english")
spacyMod = spacy.load('en_core_web_sm')


for pDf in dictSpeechesByParty.values()
    sent =' '.join([token for token in pDf['Lemmas']][0])
    dictSpeechesByParty['df_t1_Respect']['sentence'] =sent

    lemmatizedSent = spacyMod(sent)
    lemmatizedSent = [word.lemma_ for word in lemmatizedSent]
    print(len(lemmatizedSent))

    dictSpeechesByParty['df_t1_Respect']['stemmed'] = dictSpeechesByParty['df_t1_Respect']['Lemmas'].apply(lambda x: [stemmer.stem(y) for y in x])
    dictSpeechesByParty['df_t1_Respect']['lemmatized'] = [lemmatizedSent]
'''

In [ ]:
'''for i in dictSpeechesByParty['df_t1_Respect']['stemmed']:
    print (i)
    break'''

In [ ]:
dictSpeechesByParty['df_t1_Respect']

In [ ]:
%%time 
partyTimeDf = pd.DataFrame(columns = ['party', 'Lemmas', 'df_name', 'sentence', 'stems'])
for val in list(dictSpeechesByParty.values()):
    partyTimeDf = partyTimeDf.append(val)

partyTimeDf['LengthLemmas'] = partyTimeDf.Lemmas.map(len)
partyTimeDf.agg(Max=('LengthLemmas', max), Min=('LengthLemmas', 'min'), Mean=('LengthLemmas', np.mean))

In [ ]:
partyTimeDf

In [ ]:
change = ['exiting', 'seaborne', 'eurotunnel', 'withdrawal', 'departures', 'unicorn', 'remainers', 'exit', 'surrender',
          'departure', 'triggering', 'stockpiling', 'expulsion', 'blindfold', 'cliff', 'lighter', 'exits', 'triggered',
          'brexiteer', 'soft', 'plus', 'trigger', 'backroom', 'invoked', 'protesting', 'brexit', 'edge', 'canary', 
          'unicorns', 'withdrawing', 'invoking', 'withdrawn', 'manor', 'brexiteers', 'fanatics', 'postponement', 
          'currencies', 'currency', 'operability', 'operable', 'leavers', 'invoke', 'article', 'eurozone', 'clueless',
          'surrendered', 'cake', 'red', 'euroscepticism', 'prorogation', 'lining', 'gove', 'norway', 'deflationary',
          'moribund', 'eurosceptic', 'deutschmark', 'courting', 'deal', 'withdraw', 'dab', 'withdrawals', 'eurosceptics',
          'surrendering', 'aldous', 'lanarkshire', 'leaving', 'signifying', 'roofs', 'ceded', 'absentia', 'treachery',
          'dollar', 'canada', 'pragmatist', 'oven', 'ready', 'brexiters', 'control', 'capitulation', 'leave', 'referendum',
          'agreement', 'prorogue', 'smoothest', 'depreciate', 'managed', 'mutiny', 'overvalued', 'ideologues', 'foreign',
          'eec', 'war', 'prorogued', 'hannan', 'appease', 'pendolino', 'southbound', 'left', 'line', 'hard', 'bill']
 
    
#from nltk.stem import WordNetLemmatizer
 #assumes as nouns
 #pos tagging, mapping needed 

no_change = ['prime', 'even', 'parliament', 'mr', 'eu', 'bill', 'future', 'care', 'well', 'constituency', 'tax', 'children', 'uk',
             'business', 'european', 'report', 'case', 'sure', 'like', 'see', 'state', 'health', 'order', 'back', 'new', 'hope', 'local',
             'national', 'country', 'secretary', 'public', 'right', 'much', 'say', 'first', 'minister', 'labour', 'look', 'system', 'whether', 
             'members', 'million', 'good', 'today', 'services', 'clear', 'help', 'time', 'place', 'put', 'last', 'must', 'money', 'one', 
             'way', 'friend', 'work', 'would', 'think', 'two', 'great', 'could', 'lady', 'us', 'come', 'however', 'may', 'going', 'go',
             'given', 'need', 'year', 'debate', 'might', 'part', 'get', 'want', 'make', 'point', 'committee', 'years', 'also', 'know',
             'government', 'take', 'house', 'agree', 'member', 'number', 'across', 'made', 'give', 'gentleman', 'important', 'said',
             'people', 'issue', 'support', 'ensure']

stemmer = SnowballStemmer("english")
changeStem= [stemmer.stem(word) for word in change]
noChangeStem= [stemmer.stem(word) for word in no_change]


In [ ]:
partyTimeDf = partyTimeDf.reset_index()

In [ ]:
partyTimeDf.shape

# **STEMMING & LEMMATIZATION**

In [ ]:
'''%%time

# Use English stemmer.
stemmer = SnowballStemmer("english")
#lemmaz = [token.lemma_.lower() for token in doc if token.is_alpha and token.text.lower() not in stop_words]
partyTimeDf['stemmed'] = partyTimeDf['Lemmas'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
for '''

In [ ]:
%%time
# mpTimeDf['overlap2_delist'] = str(mpTimeDf['overlapping_words2'])
# *** **** **** Forceful filtering to keep brexit and leave words **** **** **** 
partyTimeDf['lemmas_delist'] = [','.join(map(str, l)) for l in partyTimeDf['Lemmas']]
#partyTimeDf = partyTimeDf[partyTimeDf['lemmas_delist'].str.contains(r'\bbrexit\b')]
#partyTimeDf = partyTimeDf[partyTimeDf['lemmas_delist'].str.contains(r'\bleave\b')]
partyTimeDf.shape

In [ ]:
%%time
overlappingWords = []
overlappingWordsN = []

for ind in partyTimeDf.index:
    partyVocabInTime = partyTimeDf.at[ind, 'stemmed']

    overlap = list(set(partyVocabInTime).intersection(changeStem))
    overlapN = list(set(partyVocabInTime).intersection(noChangeStem))

    overlappingWords.append(overlap)
    overlappingWordsN.append(overlapN)

partyTimeDf['overlapping_words'] = overlappingWords
partyTimeDf['overlapCount'] = partyTimeDf.overlapping_words.map(len)


partyTimeDf['overlapping_words_n'] = overlappingWordsN
partyTimeDf['overlapCountN'] = partyTimeDf.overlapping_words_n.map(len)


In [ ]:
partyTimeDf['overlapCountN'] = partyTimeDf.overlapping_words_n.map(len)


In [ ]:
partyTimeDf[partyTimeDf.overlapCount>45]

In [ ]:
partyTimeDf['overlap2_delist'] = [','.join(map(str, l)) for l in partyTimeDf['overlapping_words']]


In [ ]:
partyTimeDfOverridden = partyTimeDf[partyTimeDf['overlap2_delist'].str.contains('leave')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('brexit')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('remain')]
'''partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('cliff')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('exit')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('trigger')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('triggered')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('triggering')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('withdraw')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('remainers')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('bill')]'''

'''partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('control')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('hard')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('soft')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('surrender')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('prorogation')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('departure')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('backroom')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('invoked')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('edge')]'''
#partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('unicorn')]


'''
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('gentleman')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('parliament')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('us')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('come')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('however')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('may')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('soft')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('soft')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('soft')]
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('soft')]'''
partyTimeDfOverridden.shape

In [ ]:
partyTimeDf.shape


In [ ]:
%%time
partyTimeDf.to_pickle('./partyTimeDf.pkl', compression='infer', protocol=4)


In [ ]:
partyTimeDfOverridden

In [ ]:
#partyTimeDfOverridden['stemmed']=['s' for i in range(0,9)]
partyTimeDfOverridden['lemmatized']=['s' for i in range(0,9)]

In [ ]:
partyTimeDfOverridden=partyTimeDfOverridden.reset_index()
partyTimeDfOverridden.index

In [ ]:
#Commenting when using lemmatization instead of stemming
'''%%time
stemmer = SnowballStemmer("english")
colIndex = partyTimeDfOverridden.columns.get_loc('stemmed')
for index, row in partyTimeDfOverridden.iterrows():
    print(index,row['party'])
    if(len(partyTimeDfOverridden.iat[index,colIndex])>1):
        print(partyTimeDfOverridden.iat[index,colIndex][0])
        print('Already set')
        continue
    else:
       #=='s' or partyTimeDfOverridden.iat[index,colIndex].isnull()):
        print('Stemming')
        stemmed=  [stemmer.stem(y) for y in partyTimeDfOverridden.at[index,'Lemmas']] # Stem every word.
        print(len(stemmed))
        #partyTimeDfOverridden.loc[index, 'stemmed'] = stemmed
        partyTimeDfOverridden.iat[index,colIndex] = stemmed
    '''else:
        print('Already set')
        continue''''''

In [ ]:
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()
#lemma_change = [lemmatizer.lemmatize(x) for x in change]
#lemma_no_change = [lemmatizer.lemmatize(x) for x in no_change]
#print(len(change),len(set(lemma_change)))
#print(len(no_change),len(set(lemma_no_change)))

In [ ]:
%%time

colIndex = partyTimeDfOverridden.columns.get_loc('lemmatized')

for index, row in partyTimeDfOverridden.iterrows():
    print(index,row['party'])
    if(len(partyTimeDfOverridden.iat[index,colIndex])>1):
        print(partyTimeDfOverridden.iat[index,colIndex][0])
        print('Already set')
        continue
    else:
       #=='s' or partyTimeDfOverridden.iat[index,colIndex].isnull()):
        print('Lemmatizing')
        lemmed= [lemmatizer.lemmatize(y) for y in partyTimeDfOverridden.at[index,'Lemmas']]
        #[stemmer.stem(y) for y in partyTimeDfOverridden.at[index,'Lemmas']] # Stem every word.
        print(len(lemmed))
        #partyTimeDfOverridden.loc[index, 'stemmed'] = stemmed
        
        partyTimeDfOverridden.iat[index,colIndex] = lemmed
    '''else:
        print('Already set')
        continue'''

In [ ]:
len(dictSpeechesByParty.keys())

In [ ]:
listDfsKeep = partyTimeDfOverridden['df_name'].to_list()
# Dropping key-value pairs from dictionary where the key doesn't match

for k,v in list(dictSpeechesByParty.items()):
    if (k not in listDfsKeep):
        del dictSpeechesByParty[k]

len(dictSpeechesByParty.values())

In [ ]:
partyTimeDfOverridden['lemmatized']

In [ ]:
#list(partyTimeDfOverridden[partyTimeDfOverridden['df_name']=='df_t2_Labour']['stemmed'])[0]

In [ ]:
#Getting rid of this sardardi for the time being as order in dict and df seems to match

'''for k in dictSpeechesByParty.keys():
    #print(k, partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k][['df_name']])  
    #print(type(partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k]['stemmed']))
    #This creates list with faltu 0 
    #dictSpeechesByParty[k]['stemmed'] = [partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k]['stemmed']]
    print('Setting for',k)
    dictSpeechesByParty[k].at[0, 'stemmed'] = [12,2,3]
    #list(partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k]['stemmed'])[0]

    #print(len(partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k]['stemmed']))
    print('---------------------------------------------------------------------------------')

#Check
dictSpeechesByParty['df_t2_Labour (Co-op)']'''

In [ ]:
dictSpeechesByParty['df_t2_Labour (Co-op)'].dtypes
#dictSpeechesByParty['df_t2_Labour (Co-op)'].describe

In [ ]:
'''%%time
#for lemList in list(partyTimeDfOverridden['stemmed']):
    #print(lemList)
lemList = list(partyTimeDfOverridden['stemmed'])
#print(lemList[0][0])

for ind,k in enumerate(dictSpeechesByParty.keys()):
    print(ind,k)
    dictSpeechesByParty[k].at[0,'stemmed']='s'
    print(dictSpeechesByParty['df_t2_Labour (Co-op)'].dtypes)
    stInd = dictSpeechesByParty[k].columns.get_loc('stemmed')
    dictSpeechesByParty[k].iat[0,stInd]=lemList[ind]'''

In [ ]:
%%time
#for lemList in list(partyTimeDfOverridden['stemmed']):
    #print(lemList)
lemList = list(partyTimeDfOverridden['lemmatized'])
#print(lemList[0][0])

for ind,k in enumerate(dictSpeechesByParty.keys()):
    print(ind,k)
    dictSpeechesByParty[k].at[0,'lemmatized']='s'
    print(dictSpeechesByParty['df_t2_Labour (Co-op)'].dtypes)
    stInd = dictSpeechesByParty[k].columns.get_loc('lemmatized')
    dictSpeechesByParty[k].iat[0,stInd]=lemList[ind]

In [ ]:
dictSpeechesByParty['df_t2_Scottish National Party']

In [ ]:
'''for k in dictSpeechesByParty.keys():
    #print(k, partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k][['df_name']])  
    print([partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k]['stemmed']])
    dictSpeechesByParty[k]['stemmed'] = [partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k]['stemmed']]
    print(len(partyTimeDfOverridden[partyTimeDfOverridden['df_name']==k]['stemmed']))
    print('---------------------------------------------------------------------------------')'''

In [ ]:
'''%%time
stemmer = SnowballStemmer("english")

# Use English stemmer.
stemmer = SnowballStemmer("english")
#lemmaz = [token.lemma_.lower() for token in doc if token.is_alpha and token.text.lower() not in stop_words]
partyTimeDfOverridden['stemmed'] = partyTimeDf['Lemmas'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
'''

In [ ]:
partyTimeDfOverridden = partyTimeDfOverridden[partyTimeDfOverridden['overlap2_delist'].str.contains('unicorn')]
partyTimeDfOverridden.shape
partyTimeDfOverridden.shape

In [ ]:
partyTimeDf2 = partyTimeDf[partyTimeDf['overlap2_delist'].str.contains('unicorn')]
partyTimeDf2.shape

In [ ]:
'''# We should only keep speeches by those MPs which have occurences in both T1 & T2 since comparison in their vocab is to be made here

#list(vocabDf['mnis_id'].value_counts().value_counts())
#340 MPs have speeches in both T1 and T, 241 have speech(es) in one of the time periods

mpCountDict = vocabDf['mnis_id'].value_counts().to_dict()

vocabDf['mpCountInTimes'] = vocabDf['mnis_id'].map(mpCountDict)

#Drop the MPs with selected vocab in only one time interval

vocabDf=vocabDf[vocabDf['mpCountInTimes']==2]

# For manual verification'''
'''vocabDf[vocabDf['mnis_id']=='3942']
3992, 3942 - only one instance of each
4263 - 2 instances'''

In [ ]:
'''# Now backtrack and update dictionary to only keep DFs of these 921 rows

# Creating dict key names to keep from our filtered DF mnis IDs

mnis_id_list = list(vocabDf['mnis_id'].value_counts().to_dict().keys())

list1 = ['df_t1_'+item for item in mnis_id_list]
list2 = ['df_t2_'+item for item in mnis_id_list]
mnis_id_list = list1 + list2'''

In [ ]:
dictSpeechesByParty.keys()

In [ ]:
dictSpeechesByParty['df_t2_Labour (Co-op)']

In [ ]:
%%time

#  - x - x - x - CREATING & SAVING WORD2VEC MODELS FOR THE 680 MODELS - x - x - x 

dictOfModels = {}
#shutil.rmtree('./models-by-mp-and-time')
#os.makedirs('./models-by-party-and-time')
#models_folder = './models-by-party-and-time'
count = 1

for dframe in dictSpeechesByParty: 
    
# Doing in batches since notebook RAM crashe
    print(dictSpeechesByParty[dframe]['df_name'])
    print('Hello', dictSpeechesByParty[dframe]['stemmed'])
    model = gensim.models.Word2Vec(dictSpeechesByParty[dframe]['stemmed'], min_count=1, vector_size=300, window = 5, sg = 1)

    # Also saving model in a dict and exporting

    modelName ='model_'+ dframe
    print('model number', count, modelName)

    dictOfModels[dframe] = model
    #model.save(os.path.join(models_folder, modelName))
    count = count +1
    


In [ ]:
%%time


modelsToAlign = list(dictOfModels.values())
for i in range(0,len(modelsToAlign)-1):
    functools.reduce(smart_procrustes_align_gensim, modelsToAlign)

In [ ]:
dictOfModels

In [ ]:
# Check if aligned

for ind in range(0,len(listDfsKeep)-1):
    if(len(dictOfModels[listDfsKeep[ind]].wv.index_to_key)!=len(dictOfModels[listDfsKeep[ind+1]].wv.index_to_key)):
        print('Vocabs not similar')
        
print('Vocab Size', len(dictOfModels[listDfsKeep[ind]].wv.index_to_key))

#Yes Aligned, 5384 Vocab size
# Vocab Size 3686 after stemming
# Vocab Size after mennatizing = 4844

#Vocab size 3606 after stemming and only considering common words - brexit, leave, remain

In [ ]:
len(set(dictOfModels[listDfsKeep[ind]].wv.index_to_key).intersection(changeStem))
   # overlap = list(set(partyVocabInTime).intersection(changeStem))


In [ ]:
len(set(dictOfModels[listDfsKeep[ind]].wv.index_to_key).intersection(noChangeStem))


In [ ]:
'''# To delete if need be
import shutil
shutil.rmtree('./models-by-party-and-time')

os.remove('./file.zip')'''

In [ ]:
%%time
os.makedirs('./models-by-party-and-time')
models_folder = './models-by-party-and-time'

for k in dictOfModels.keys(): 
    dictOfModels[k].save(os.path.join(models_folder, k))


In [ ]:
'''%%time
# Commenting since we're going to load and start with alignment of models with a fresh notebook
functools.reduce(smart_procrustes_align_gensim, list(dictOfModels.values()))'''

In [ ]:
%%time
!zip -r file.zip /kaggle/working/models-by-party-and-time

# **OPTIONAL SIMILARITY CHECK, CAN SKIP**

In [ ]:
dictOfModels
for i in dictOfModels.keys():
  print(dictOfModels[i], dictOfModels[i].wv.similar_by_word('brexit', 10))
  print('- x - x - x - x - x - x - x - x - x')